In [96]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

label_encoder =LabelEncoder()
data = '../data/WASSA23_conv_level_with_labels_train.tsv'

xy = pd.read_table(data, header=0)
new_col = []
for names in xy.columns:
    new_col.append(names.strip())
xy.columns = new_col
xy = xy.loc[:, ['text','EmotionalPolarity']]
xy['EmotionalPolarity'] = label_encoder.fit_transform(xy['EmotionalPolarity'])
# y['Emotion'] = label_encoder.fit_transform(y['Emotion'])
# y['Empathy'] = label_encoder.fit_transform(y['Empathy'])
# print(y)
# y = y['EmotionalPolarity']

In [97]:
xy.shape[0]

8776

In [98]:
from torch.utils.data import Dataset, DataLoader, random_split
class EEPD_Dataset(Dataset):
	def __init__(self, df):
		
		self.text = [text for text in df.loc[:, 'text']]
		self.label = [label for label in df.loc[:, 'EmotionalPolarity']]
		self.n_samples = df.shape[0]

	def __getitem__(self, index):
		return self.text[index], self.label[index]

	def __len__(self):
		return self.n_samples

In [99]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
data = EEPD_Dataset(xy)

tokenizer = get_tokenizer("basic_english")
# train_iter = AG_NEWS(split="train")


def yield_tokens(data_iter):
    for text, _ in data_iter:
        yield tokenizer(text)


vocab = build_vocab_from_iterator(yield_tokens(data), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [100]:
data[11]

('Yup! Well it was nice talking to you! bye                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ',
 1)

In [106]:
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)
def collate_batch(batch):
    text_list, label_list, offsets = [], [], [0]
    for _text, _label  in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), label_list.to(device), offsets.to(device)

In [130]:
from torch import nn


class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc1 = (nn.Linear(embed_dim, embed_dim))
        self.fc2 = (nn.Linear(embed_dim, num_class))
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        out = self.fc1(embedded)
        return self.fc2(out)

In [131]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = torch.Generator().manual_seed(42)

train_set, test_set, valid_set = random_split(dataset=data, lengths=[0.8, 0.1, 0.1], generator=generator)

num_class = len(set([label for (text, label) in train_set]))
vocab_size = len(vocab)
emsize = 128
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [132]:
import time


def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (text, label, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = criterion(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print(
                "| epoch {:3d} | {:5d}/{:5d} batches "
                "| accuracy {:8.3f}".format(
                    epoch, idx, len(dataloader), total_acc / total_count
                )
            )
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (text, label, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = criterion(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc / total_count

In [133]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset

# Hyperparameters
EPOCHS = 20  # epoch
LR = 0.01  # learning rate
BATCH_SIZE = 64  # batch size for training

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None

train_dataloader =dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
valid_dataloader = dataloader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)
test_dataloader = dataloader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print("-" * 59)
    print(
        "| end of epoch {:3d} | time: {:5.2f}s | "
        "valid accuracy {:8.3f} ".format(
            epoch, time.time() - epoch_start_time, accu_val
        )
    )
    print("-" * 59)

-----------------------------------------------------------
| end of epoch   1 | time:  3.72s | valid accuracy    0.368 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time:  2.68s | valid accuracy    0.328 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time:  2.68s | valid accuracy    0.347 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time:  2.56s | valid accuracy    0.343 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time:  2.38s | valid accuracy    0.343 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time:  2.54s |

In [125]:
print("Checking the results of test dataset.")
accu_test = evaluate(test_dataloader)
print("test accuracy {:8.3f}".format(accu_test))

Checking the results of test dataset.
test accuracy    0.377
